# Data Wrangling
__Math 3080: Fundamentals of Data Science__

Reading:
* [McKinney, *Python for Data Science*, Chapter 6](https://wesmckinney.com/book/accessing-data)
* Chapters 8,10

Class notes are found through GitHub. As changes are made, they will automatically be uploaded to GitHub. A link to the repository is on Canvas.

-----
## Outcomes
### Course

### Data Analysis Certification
* Databases: Data joins

-----
## Outline
* Concatenating
* Mapping
* Encoding
    * One-hot encoding / Dummy Variables
    * Ordinal Encoding
* Joining
* Reshaping

What is the purpose of data wrangling?
* Combine datasets that augment our view of the data
* Look at the data from a different perspective
* ... other reasons ...

This is one of the big skills of a Data Scientiest

In [1]:
import pandas as pd
students = pd.read_excel('../Datasets/Gradebook.xlsx', sheet_name='Students')
assignments = pd.read_excel('../Datasets/Gradebook.xlsx', sheet_name='Assignments')
grades = pd.read_excel('../Datasets/Gradebook.xlsx', sheet_name='Grades')


In [2]:
print(students.shape)
students.head()

(50, 10)


,student_id,first_name,last_name,class_year,gpa,major,first_generation_student,financial_aid,housing_status,credits_completed
0,1001,Sarah,Jensen,Sophomore,2.87,History,Yes,Yes,On-Campus,79
1,1002,Marcus,Lee,Junior,3.63,Physics,No,Yes,Off-Campus,83
2,1003,Emily,Torres,Junior,3.27,Biology,No,Yes,Off-Campus,51
3,1004,Daniel,Wright,Junior,2.85,English,Yes,No,On-Campus,58
4,1005,Olivia,Patel,Sophomore,3.21,Business,No,Yes,Off-Campus,26


In [3]:
print(assignments.shape)
assignments.head()

(23, 3)


,assignment_id,assignment_name,category
0,A1,Homework 1,Homework
1,A2,Homework 2,Homework
2,A3,Homework 3,Homework
3,A4,Homework 4,Homework
4,A5,Homework 5,Homework


In [4]:
print(grades.shape)
grades.head()

(1150, 3)


,student_id,assignment_id,grade
0,1001,A1,100
1,1035,A1,100
2,1002,A1,99
3,1030,A1,98
4,1021,A1,97


## Concatenate data

Let's say we have a couple new students move in. We need to add their data to the students dataframe.

In [5]:
new_students = pd.DataFrame({
    'student_id' : [1101, 1102, 1103],
    'first_name' : ['John', 'Jill', 'Jim'],
    'last_name' : ['Anderson', 'Benson', 'Kent'],
    'class_year' : ['Senior', 'Freshman', 'Sophomore'],
    'gpa' : [3.41, 2.77, 3.87],
    'major' : ['Physics', 'Biology', 'English'],
    'first_generation_student' : ['Yes', 'Yes', 'Yes'],
    'financial_aid' : ['Yes', 'Yes', 'Yes'],
    'housing_status' : ['On-Campus', 'On-Campus', 'Off-Campus'],
    'credits_completed' : [82, 7, 16]
})

new_students

,student_id,first_name,last_name,class_year,gpa,major,first_generation_student,financial_aid,housing_status,credits_completed
0,1101,John,Anderson,Senior,3.41,Physics,Yes,Yes,On-Campus,82
1,1102,Jill,Benson,Freshman,2.77,Biology,Yes,Yes,On-Campus,7
2,1103,Jim,Kent,Sophomore,3.87,English,Yes,Yes,Off-Campus,16


In [6]:
pd.concat([students, new_students], axis=0).tail(7)

,student_id,first_name,last_name,class_year,gpa,major,first_generation_student,financial_aid,housing_status,credits_completed
46,1047,Brooklyn,Ward,Sophomore,2.54,Engineering,Yes,No,Off-Campus,86
47,1048,Thomas,Hughes,Freshman,2.77,Engineering,Yes,No,On-Campus,72
48,1049,Hailey,Adams,Freshman,2.67,Chemistry,No,Yes,Off-Campus,94
49,1050,Benjamin,Wright,Senior,3.96,Chemistry,No,Yes,Off-Campus,49
0,1101,John,Anderson,Senior,3.41,Physics,Yes,Yes,On-Campus,82
1,1102,Jill,Benson,Freshman,2.77,Biology,Yes,Yes,On-Campus,7
2,1103,Jim,Kent,Sophomore,3.87,English,Yes,Yes,Off-Campus,16


(What if we don't have data on all variables? Try commenting a couple of rows from the `new_students` dictionary.)

Now, let's say that in the assignments folder, we want to identify which segment of the course it is assigned in. Let's say the course is broken into 4 segments. We can do this in one of three ways: (1) concatenating, (2) directly adding columns, and (3) mapping.

In [7]:
segments = pd.Series([1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,1,1,2,3,3,4,2,4])
pd.concat([assignments, segments], axis=1)

,assignment_id,assignment_name,category,0
0,A1,Homework 1,Homework,1
1,A2,Homework 2,Homework,1
2,A3,Homework 3,Homework,1
3,A4,Homework 4,Homework,2
4,A5,Homework 5,Homework,2
5,A6,Homework 6,Homework,2
6,A7,Homework 7,Homework,2
7,A8,Homework 8,Homework,3
8,A9,Homework 9,Homework,3
9,A10,Homework 10,Homework,3


In [8]:
assignments['segment'] = segments
assignments

,assignment_id,assignment_name,category,segment
0,A1,Homework 1,Homework,1
1,A2,Homework 2,Homework,1
2,A3,Homework 3,Homework,1
3,A4,Homework 4,Homework,2
4,A5,Homework 5,Homework,2
5,A6,Homework 6,Homework,2
6,A7,Homework 7,Homework,2
7,A8,Homework 8,Homework,3
8,A9,Homework 9,Homework,3
9,A10,Homework 10,Homework,3


## Mapping

Let's map the assignment type from the assignments sheet onto the grades sheet.

In [9]:
hw_dict = pd.Series(assignments['category'].values, index=assignments['assignment_id']).to_dict()
hw_dict

{'A1': 'Homework',
 'A2': 'Homework',
 'A3': 'Homework',
 'A4': 'Homework',
 'A5': 'Homework',
 'A6': 'Homework',
 'A7': 'Homework',
 'A8': 'Homework',
 'A9': 'Homework',
 'A10': 'Homework',
 'A11': 'Homework',
 'A12': 'Homework',
 'A13': 'Homework',
 'A14': 'Homework',
 'A15': 'Homework',
 'P1': 'Computer Project',
 'P2': 'Computer Project',
 'P3': 'Computer Project',
 'P4': 'Computer Project',
 'P5': 'Computer Project',
 'P6': 'Computer Project',
 'E1': 'Exam',
 'E2': 'Exam'}

In [10]:
grades['category'] = grades['assignment_id'].map(hw_dict)
grades.head()

,student_id,assignment_id,grade,category
0,1001,A1,100,Homework
1,1035,A1,100,Homework
2,1002,A1,99,Homework
3,1030,A1,98,Homework
4,1021,A1,97,Homework


## One-hot Encoding

When sending data into a model, it will need numerical variables. So, we need to convert categorical variables into numerical variables that represent those categories. This process is known as __encoding__.

For Nominal variables, we use __one-hot encoding__ (in Pandas, we use the `pd.get_dummies()` function to do this). It creates one column for each category and then gives it a value of 1 if the observation is in that category and a 0 if it is not.

Let's do this for the students' major in the `students` dataset.

In [11]:
students.head()

,student_id,first_name,last_name,class_year,gpa,major,first_generation_student,financial_aid,housing_status,credits_completed
0,1001,Sarah,Jensen,Sophomore,2.87,History,Yes,Yes,On-Campus,79
1,1002,Marcus,Lee,Junior,3.63,Physics,No,Yes,Off-Campus,83
2,1003,Emily,Torres,Junior,3.27,Biology,No,Yes,Off-Campus,51
3,1004,Daniel,Wright,Junior,2.85,English,Yes,No,On-Campus,58
4,1005,Olivia,Patel,Sophomore,3.21,Business,No,Yes,Off-Campus,26


In [12]:
major_dummies = pd.get_dummies(students['major']).astype(int)
major_dummies.head()

,Biology,Business,Chemistry,Computer Science,Engineering,English,History,Mathematics,Physics,Psychology,Sociology,Statistics
0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0


How do we add this onto the original dataset? We need to concatenate them.

In [13]:
students = pd.concat([students, major_dummies], axis=1)
students.head(7)

,student_id,first_name,last_name,class_year,gpa,major,first_generation_student,financial_aid,housing_status,credits_completed,...,Chemistry,Computer Science,Engineering,English,History,Mathematics,Physics,Psychology,Sociology,Statistics
0,1001,Sarah,Jensen,Sophomore,2.87,History,Yes,Yes,On-Campus,79,...,0,0,0,0,1,0,0,0,0,0
1,1002,Marcus,Lee,Junior,3.63,Physics,No,Yes,Off-Campus,83,...,0,0,0,0,0,0,1,0,0,0
2,1003,Emily,Torres,Junior,3.27,Biology,No,Yes,Off-Campus,51,...,0,0,0,0,0,0,0,0,0,0
3,1004,Daniel,Wright,Junior,2.85,English,Yes,No,On-Campus,58,...,0,0,0,1,0,0,0,0,0,0
4,1005,Olivia,Patel,Sophomore,3.21,Business,No,Yes,Off-Campus,26,...,0,0,0,0,0,0,0,0,0,0
5,1006,Jason,Kim,Freshman,3.11,History,No,Yes,Off-Campus,66,...,0,0,0,0,1,0,0,0,0,0
6,1007,Hannah,Brooks,Senior,3.77,Engineering,Yes,No,Off-Campus,78,...,0,0,1,0,0,0,0,0,0,0


## Ordinal Encoding

For Ordinal variables, we can depict the category with a number scale (for example, pain level is often represented "on a scale from 1 to 10"). We can easily use this with mapping on our class level for each student.

In [14]:
classes = {
    'Freshman' : 1,
    'Sophomore' : 2,
    'Junior' : 3,
    'Senior' : 4
}

students['class_num'] = students['class_year'].map(classes)
students.head(7)

,student_id,first_name,last_name,class_year,gpa,major,first_generation_student,financial_aid,housing_status,credits_completed,...,Computer Science,Engineering,English,History,Mathematics,Physics,Psychology,Sociology,Statistics,class_num
0,1001,Sarah,Jensen,Sophomore,2.87,History,Yes,Yes,On-Campus,79,...,0,0,0,1,0,0,0,0,0,2
1,1002,Marcus,Lee,Junior,3.63,Physics,No,Yes,Off-Campus,83,...,0,0,0,0,0,1,0,0,0,3
2,1003,Emily,Torres,Junior,3.27,Biology,No,Yes,Off-Campus,51,...,0,0,0,0,0,0,0,0,0,3
3,1004,Daniel,Wright,Junior,2.85,English,Yes,No,On-Campus,58,...,0,0,1,0,0,0,0,0,0,3
4,1005,Olivia,Patel,Sophomore,3.21,Business,No,Yes,Off-Campus,26,...,0,0,0,0,0,0,0,0,0,2
5,1006,Jason,Kim,Freshman,3.11,History,No,Yes,Off-Campus,66,...,0,0,0,1,0,0,0,0,0,1
6,1007,Hannah,Brooks,Senior,3.77,Engineering,Yes,No,Off-Campus,78,...,0,1,0,0,0,0,0,0,0,4


-----

## Joins

A __join__ is where you pair observations from one table with observations from another table. Let's look at these two tables:

In [15]:
import numpy as np
import pandas as pd

depts = pd.DataFrame({
    'dept_id' : [10,20,30,40],
    'dept_name' : ['Engineering','Sales','Marketing','HR']
})

display(depts)

employees = pd.DataFrame({
    'emp_id' : [1,2,3,4,5],
    'emp_name' : ['Alice','Bob','Carol','Dan','Eve'],
    'dept_id' : [10,20,20,50,np.nan]
})

display(employees)

,dept_id,dept_name
0,10,Engineering
1,20,Sales
2,30,Marketing
3,40,HR


,emp_id,emp_name,dept_id
0,1,Alice,10.0
1,2,Bob,20.0
2,3,Carol,20.0
3,4,Dan,50.0
4,5,Eve,NaN


There are 4 ways we can join tables:

### Left Join

To call a join, you need to list two tables. A __left join__ will take all the entries for the first table. If there are matches in the second table, it will include that data in another column. If not, it will mark the missing data as `NaN`.

<img src="./images/leftjoin.png" alt="Left Join" width=250, height=250>

In [16]:
pd.merge(employees, depts, on='dept_id', how='left')

,emp_id,emp_name,dept_id,dept_name
0,1,Alice,10.0,Engineering
1,2,Bob,20.0,Sales
2,3,Carol,20.0,Sales
3,4,Dan,50.0,NaN
4,5,Eve,NaN,NaN


### Right Join

A __right join__ will take all the entries for the second table. If there are matches in the first table, it will include that data in another column. If not, it will mark the missing data as `NaN`.

<img src="./images/rightjoin.png" alt="Right Join" width=250, height=250>

In [17]:
pd.merge(employees, depts, on='dept_id', how='right')

,emp_id,emp_name,dept_id,dept_name
0,1.0,Alice,10.0,Engineering
1,2.0,Bob,20.0,Sales
2,3.0,Carol,20.0,Sales
3,NaN,NaN,30.0,Marketing
4,NaN,NaN,40.0,HR


### Outer Join

An __outer join__ will take all the entries for both tables. Any missing data is marked as `NaN`.

<img src="./images/outerjoin.png" alt="Outer Join" width=250, height=250>

In [18]:
pd.merge(employees, depts, on='dept_id', how='outer')

,emp_id,emp_name,dept_id,dept_name
0,1.0,Alice,10.0,Engineering
1,2.0,Bob,20.0,Sales
2,3.0,Carol,20.0,Sales
3,NaN,NaN,30.0,Marketing
4,NaN,NaN,40.0,HR
5,4.0,Dan,50.0,NaN
6,5.0,Eve,NaN,NaN


### Inner Join

An __inner join__ will take only entries that are found in both tables. 

<img src="./images/innerjoin.png" alt="Inner Join" width=250, height=250>

In [19]:
pd.merge(employees, depts, on='dept_id', how='inner')

,emp_id,emp_name,dept_id,dept_name
0,1,Alice,10.0,Engineering
1,2,Bob,20.0,Sales
2,3,Carol,20.0,Sales


### Note on column names during joins
Sometimes, the column name used for merging is different. Let's assume that in the employees dataframe, the department was indicated differently.

In [20]:
display(depts)

employees = pd.DataFrame({
    'emp_id' : [1,2,3,4,5],
    'emp_name' : ['Alice','Bob','Carol','Dan','Eve'],
    'department' : [10,20,20,50,np.nan]
})

display(employees)

,dept_id,dept_name
0,10,Engineering
1,20,Sales
2,30,Marketing
3,40,HR


,emp_id,emp_name,department
0,1,Alice,10.0
1,2,Bob,20.0
2,3,Carol,20.0
3,4,Dan,50.0
4,5,Eve,NaN


The department is still how we want to do the join. However, we have to indicate each column specifically since they are not under the same name.

In [21]:
pd.merge(employees, depts, left_on='department', right_on='dept_id', how='inner')

,emp_id,emp_name,department,dept_id,dept_name
0,1,Alice,10.0,10,Engineering
1,2,Bob,20.0,20,Sales
2,3,Carol,20.0,20,Sales


-----

## Reshaping

Reshaping dataframes involves reorganizing data to change how variables are arranged without altering the underlying values. This will present the data is a more suitable format for visualization and summarization.

Most data is recorded in long format. With __long-format data__, variables are stacked into a single column with corresponding value and identifier columns.

(With our gradbook dataset, the category is listed in the `assignment_id` column with the value in the `grade` column.)

In [22]:
import pandas as pd
grades = pd.read_excel('../Datasets/Gradebook.xlsx', sheet_name='Grades')
grades.head()

,student_id,assignment_id,grade
0,1001,A1,100
1,1035,A1,100
2,1002,A1,99
3,1030,A1,98
4,1021,A1,97


For visualization, we often change long-format data into wide format. With __wide-format data__, each variable has its own column and observations are spread across many columns.

(With our gradebook dataset, each category has its own column and each element represents the value for the particular identifier and category.)

In [23]:
students = pd.read_excel('../Datasets/Gradebook.xlsx', sheet_name='Students')
students.head()

,student_id,first_name,last_name,class_year,gpa,major,first_generation_student,financial_aid,housing_status,credits_completed
0,1001,Sarah,Jensen,Sophomore,2.87,History,Yes,Yes,On-Campus,79
1,1002,Marcus,Lee,Junior,3.63,Physics,No,Yes,Off-Campus,83
2,1003,Emily,Torres,Junior,3.27,Biology,No,Yes,Off-Campus,51
3,1004,Daniel,Wright,Junior,2.85,English,Yes,No,On-Campus,58
4,1005,Olivia,Patel,Sophomore,3.21,Business,No,Yes,Off-Campus,26


Wide format is more useful for data analysis and preparation. However, we often prefer to store data in long format because it allows for more flexibility with how the data is used later. Let's learn how to move between long and wide formats.

### Aggregation

In order to reshape the data using a Groupby, we are going to need a method to summarize the values in a dataset. For instance, if we have a table with the student names for rows and homework, project, and exam grades as columns, the values in the table have to summarize and represent all the homework, project, and exam entries for that student. These summary values are found using __aggregate functions__.

Common aggregate functions:
```python
agg('max')
agg('min')
agg('mean') # Default
agg('std')
agg('count')
```

We can even create our own aggregate function
```python    
def range(x):
    return x.max()-x.min()

agg(range)
```

In [24]:
# Using one aggregate function
print(students['gpa'].mean())

print(students['gpa'].aggregate(['mean', 'min', 'median', 'max', 'std']))

3.2092
mean      3.209200
min       2.500000
median    3.175000
max       3.960000
std       0.461134
Name: gpa, dtype: float64


We'll see these aggregate functions in use in the following topics.


### Pivot Tables

We start with a table in long format. 
* Choose one variable to be the row in our table
* Choose another variable to be the column in our table
* Choose a third variable to be the value in the table

If there is more than one value to go into the table, what do we do? This is where we use our aggregate function.

In [25]:
grade_table = pd.pivot_table(grades, index='student_id', columns='assignment_id', values='grade', aggfunc='max')
grade_table

assignment_id,A1,A10,A11,A12,A13,A14,A15,A2,A3,A4,...,A8,A9,E1,E2,P1,P2,P3,P4,P5,P6
student_id,,,,,,,,,,,,,,,,,,,,,
1001,100,69,99,79,78,95,65,70,60,95,...,70,62,91,88,67,80,93,77,66,82
1002,99,63,71,97,87,93,63,71,66,87,...,97,93,79,60,74,83,90,73,65,84
1003,70,62,75,95,65,73,82,74,89,60,...,76,76,82,69,83,96,80,99,73,77
1004,68,72,60,76,81,73,91,81,68,74,...,68,75,63,92,72,69,80,70,89,95
1005,94,85,77,95,81,72,100,91,95,97,...,64,61,67,79,87,90,96,80,66,69
1006,75,88,75,78,88,63,98,65,79,62,...,70,99,69,81,71,63,98,95,86,75
1007,77,76,97,92,92,91,94,78,68,65,...,90,87,96,63,71,70,65,87,64,96
1008,79,100,77,100,87,62,79,91,98,100,...,66,73,86,90,66,97,97,84,79,83
1009,78,68,90,87,64,85,67,76,74,95,...,89,99,69,67,95,78,77,72,73,75


### Melting

Melting is the opposite of a pivot table. It takes the row of your table and makes that one variable, the column of your table becomes another variable, and the value becomes a third variable.

In [26]:
grade_table.melt(ignore_index=False)

,assignment_id,value
student_id,,
1001,A1,100
1002,A1,99
1003,A1,70
1004,A1,68
1005,A1,94
...,...,...
1046,P6,69
1047,P6,66
1048,P6,65


### Groupbys

In [27]:
students.groupby('class_year')['gpa'].mean()

class_year
Freshman     2.840000
Junior       3.334545
Senior       3.359286
Sophomore    3.223333
Name: gpa, dtype: float64

In [30]:
students.groupby(['class_year','major'])['gpa'].mean()

class_year  major           
Freshman    Biology             2.700
            Business            3.020
            Chemistry           2.670
            Engineering         2.890
            History             3.110
            Physics             2.920
            Psychology          2.540
            Sociology           2.740
Junior      Biology             3.070
            Business            3.760
            Chemistry           3.810
            Engineering         2.630
            English             2.850
            History             3.260
            Physics             3.630
            Psychology          3.090
            Statistics          3.700
Senior      Biology             3.195
            Chemistry           3.420
            Computer Science    2.770
            Engineering         3.740
            English             3.080
            History             3.850
            Physics             3.520
            Psychology          3.790
            Statistic

In [ ]:
students.groupby(['major','class_year'])['gpa'].mean()

major             class_year
Biology           Freshman      2.700
                  Junior        3.070
                  Senior        3.195
                  Sophomore     3.300
Business          Freshman      3.020
                  Junior        3.760
                  Sophomore     3.210
Chemistry         Freshman      2.670
                  Junior        3.810
                  Senior        3.420
                  Sophomore     3.620
Computer Science  Senior        2.770
                  Sophomore     3.840
Engineering       Freshman      2.890
                  Junior        2.630
                  Senior        3.740
                  Sophomore     2.695
English           Junior        2.850
                  Senior        3.080
                  Sophomore     3.470
History           Freshman      3.110
                  Junior        3.260
                  Senior        3.850
                  Sophomore     2.870
Mathematics       Sophomore     2.500
Physics           Fre